In [ ]:
np.random.seed(100)
random_rows = np.random.choice(train.index,10000)
train_selected = train.iloc[random_rows]
train_selected.to_csv('data/train_selected.csv',index=False)

In [10]:
def check_na(df):
    ans = []
    for k in df.keys():
        if True in df[k].isnull().values:
            ans.append(k)
    return ans

#na_of_train = check_na(train)
#na_of_test = check_na(test)
    

In [ ]:
na_of_train

In [ ]:
na_of_test

In [ ]:
%matplotlib

In [ ]:
important_feature = np.argsort(forest.feature_importances_)[::-1]
labels = []
num = []
for i in important_feature:
    labels.append(x.keys()[i])
    num.append(forest.feature_importances_[i])

In [ ]:
for nn in num:
    print nn
for ll in labels:
    print ll

In [ ]:
fig, ax = plt.subplots()
#plt.bar(range(30),num[:30])
ax.hist(num[:30], range(30),orientation='horizontal')
#ax.set_xticklabels(labels[:30])
ax.set_ylim([0, 30])


plt.show()

In [ ]:
important_feature.sort()

In [ ]:
f = plt.figure()
forest.feature_importances_
plt.plot(range(1,271),forest.feature_importances_)
plt.show()

In [ ]:
important_feature = np.argsort(forest.feature_importances_)[::-1]
f = open('data/feature_importance.txt','w')
for ind in important_feature:
    print x.keys()[ind] + ': ' + str(forest.feature_importances_[ind])
    print >> f, x.keys()[ind]


In [ ]:
from collections import OrderedDict
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
RANDOM_STATE = 123

# Generate a binary classification dataset.
#X, y = make_classification(n_samples=500, n_features=25,
#                           n_clusters_per_class=1, n_informative=15,
#                           random_state=RANDOM_STATE)

# NOTE: Setting the `warm_start` construction parameter to `True` disables
# support for paralellised ensembles but is necessary for tracking the OOB
# error trajectory during training.
ensemble_clfs = [
    ("RandomForestClassifier, max_features='sqrt'",
        RandomForestClassifier(warm_start=True, oob_score=True,
                               max_features="sqrt",
                               random_state=RANDOM_STATE)),
    ("RandomForestClassifier, max_features='log2'",
        RandomForestClassifier(warm_start=True, max_features='log2',
                               oob_score=True,
                               random_state=RANDOM_STATE)),
    ("RandomForestClassifier, max_features=None",
        RandomForestClassifier(warm_start=True, max_features=None,
                               oob_score=True,
                               random_state=RANDOM_STATE))
]

# Map a classifier name to a list of (<n_estimators>, <error rate>) pairs.
error_rate = OrderedDict((label, []) for label, _ in ensemble_clfs)

# Range of `n_estimators` values to explore.
min_estimators = 15
max_estimators = 175

for label, clf in ensemble_clfs:
    for i in range(min_estimators, max_estimators + 1):
        clf.set_params(n_estimators=i)
        clf.fit(np.array(x),np.array(y))

        # Record the OOB error for each `n_estimators=i` setting.
        oob_error = 1 - clf.oob_score_
        error_rate[label].append((i, oob_error))

# Generate the "OOB error rate" vs. "n_estimators" plot.
for label, clf_err in error_rate.items():
    xs, ys = zip(*clf_err)
    plt.plot(xs, ys, label=label)

plt.xlim(min_estimators, max_estimators)
plt.xlabel("n_estimators")
plt.ylabel("OOB error rate")
plt.legend(loc="upper right")
plt.show()

from sklearn.ensemble import RandomForestClassifier
params = {'warm_start':True, 'oob_score':True}
p = [{'max_features':'sqrt'},{'max_features':'log2'},{'max_features':None}]

label= ["max_features='sqrt'","max_features='log2'","max_features=None"]
num_of_estimator = range(300)[20:300:5]
error = {x:[] for x in label}

for i, l in enumerate(label):
    for n in num_of_estimator:
        p[i]['n_estimators']=n
        params.update(p[i])
        rf = RandomForestClassifier(**params)
        rf.fit(np.array(x),np.array(y))
        error[l].append(1 - rf.oob_score_)


In [ ]:
pd.DataFrame(error).to_csv('data/maxfeature_nestimator_oob.csv',index=False)

In [ ]:
for k in error.keys():
    plt.plot(num_of_estimator, error[k], label=k)
        
plt.xlabel("n_estimators")
plt.ylabel("OOB error rate")
plt.legend(loc='best')
#plt.show()
plt.savefig('/Users/itbwtw/Documents/Courses/CompSci 273A Machine Learning/project/latex/figure/oob.eps')

In [23]:
f = open('data/feature_importance.txt','r')
features = []
scores = []
for line in f:
    features.append(line.strip())
len(features)
for ii in range(2,100):
    rf = RandomForestClassifier(n_estimators = 100,oob_score=True)
    rf = rf.fit(np.array(x[features[0:ii]]),np.array(y))
    scores.append(rf.oob_score_)
    print ii


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [21]:
plt.figure()
plt.plot(range(2,30),scores)
plt.xlabel('Size of Feature Dimension')
plt.ylabel('Area Under ROC curve')

In [22]:
plt.savefig('/Users/itbwtw/Documents/Courses/CompSci 273A Machine Learning/project/latex/figure/feat.eps')

In [ ]:
plt.show(1)

In [ ]:
plt.figure(2)
plt.plot(range(2,21),scores)

In [ ]:
plt.show()

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
train_file = 'data/train_selected.csv'
test_file = 'data/test.csv'
train = pd.read_csv(train_file)
test = pd.read_csv(test_file)
train.head()

,QuoteNumber,Original_Quote_Date,QuoteConversion_Flag,Field6,Field7,Field8,Field9,Field10,Field11,Field12,...,GeographicField59A,GeographicField59B,GeographicField60A,GeographicField60B,GeographicField61A,GeographicField61B,GeographicField62A,GeographicField62B,GeographicField63,GeographicField64
0,63850,2013-07-29,0,B,24,0.9403,0.0006,965,1.0200,N,...,19,23,-1,20,-1,14,-1,8,N,CA
1,93374,2014-07-01,0,E,16,0.9392,0.0006,"1,487",1.3045,N,...,25,25,-1,15,-1,5,-1,25,N,IL
2,351241,2014-10-27,1,E,16,0.9108,0.0006,"1,487",1.3045,N,...,15,18,-1,11,-1,20,-1,21,N,IL
3,422808,2014-04-16,0,B,23,0.9153,0.0007,935,1.0200,N,...,15,19,-1,16,-1,19,-1,14,N,CA
4,129368,2013-08-09,0,B,24,0.9403,0.0006,965,1.0200,N,...,25,25,-1,12,-1,20,-1,8,N,CA


In [3]:
keys = train.keys()
dtype = train.dtypes
not_number = []
i = 0
for k in keys:
    if (str(dtype[i]) != 'float64') & (str(dtype[i]) != 'int64'):
        not_number.append(k)
    i+=1
train[not_number].head()

,Original_Quote_Date,Field6,Field10,Field12,CoverageField8,CoverageField9,SalesField7,PersonalField7,PersonalField16,PersonalField17,...,PropertyField30,PropertyField31,PropertyField32,PropertyField33,PropertyField34,PropertyField36,PropertyField37,PropertyField38,GeographicField63,GeographicField64
0,2013-07-29,B,965,N,T,J,K,N,ZA,ZE,...,N,O,Y,H,N,N,N,N,N,CA
1,2014-07-01,E,"1,487",N,Y,A,P,N,XB,XH,...,N,O,Y,H,Y,N,N,N,N,IL
2,2014-10-27,E,"1,487",N,T,A,K,N,XS,XG,...,N,O,Y,H,Y,N,N,N,N,IL
3,2014-04-16,B,935,N,Y,D,T,N,ZA,ZE,...,N,O,Y,H,Y,N,N,N,N,CA
4,2013-08-09,B,965,N,T,D,V,N,ZA,ZE,...,N,N,Y,G,Y,N,N,Y,N,CA


In [4]:
not_number.pop(not_number.index('Original_Quote_Date'))
not_number.pop(not_number.index('Field10'))

'Field10'

In [5]:
from sklearn import preprocessing

for k in not_number:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(np.unique(list(train[k].values) + list(test[k].values)))
    train[k] = lbl.transform(list(train[k].values))
    test[k] = lbl.transform(list(test[k].values))

In [6]:
def convert_date(df):
    df['year'] = df['Original_Quote_Date'].apply(lambda x : x[0:4]).astype('int')
    df['month'] = df['Original_Quote_Date'].apply(lambda x : x[5:7]).astype('int')
    df['day'] = df['Original_Quote_Date'].apply(lambda x : x[8:]).astype('int')
def convert_field10(df):
    df['Field10'] = df['Field10'].apply(lambda x: str(x).replace(',','')).astype('int')
def fill_PersonalField84(df):
    df.loc[df['PersonalField84'].isnull(),'PersonalField84'] = 2
def fill_PropertyField29(df):
    df.loc[df['PropertyField29'].isnull(),'PropertyField29'] = df['PropertyField29'].mean()


In [11]:
convert_date(train)
convert_field10(train)
convert_date(test)
convert_field10(test)
y = train['QuoteConversion_Flag']
x = train.select_dtypes(include = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64'])
xt = test.select_dtypes(include = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64'])
x = x.drop('QuoteConversion_Flag',axis= 1)
print check_na(x), check_na(xt)
print x.shape, xt.shape

['PersonalField84', 'PropertyField29'] ['PersonalField84', 'PropertyField29']
(10000, 300) (173836, 300)


In [12]:
fill_PersonalField84(x)
fill_PersonalField84(xt)
fill_PropertyField29(x)
fill_PropertyField29(xt)

/Users/itbwtw/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:415: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [ ]:
selected_feature = []
f = open('data/feature_importance.txt', 'r')
i = 0
for line in f:
    if i >= 22:
        break
    else:
        i+=1
    selected_feature.append(line.strip())

x = x[selected_feature]

In [13]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100,oob_score=True)
forest = forest.fit(np.array(x),np.array(y))

In [ ]:
forest.oob_score_ #300

In [ ]:
yhat = forest.predict_proba(np.array(xt))

In [ ]:
yhat = yhat[:,1]

In [ ]:
from sklearn import ensemble
params = {'n_estimators': 10, 'max_leaf_nodes': 4, 'max_depth': None, 'random_state': 2,
          'min_samples_split': 5, 'learning_rate': 0.1, 'subsample': 0.5}
gb = ensemble.GradientBoostingClassifier(**params)
gb.fit(x, y)

In [ ]:
yhat = gb.predict_proba(xt)

In [ ]:
result_data = {'QuoteNumber':xt['QuoteNumber'],'QuoteConversion_Flag':yhat[:,1]}

In [ ]:
result = pd.DataFrame(result_data,columns=('QuoteNumber','QuoteConversion_Flag'))

In [ ]:
result.to_csv('data/result.csv',index=False)

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.pipeline import make_pipeline
from sklearn import linear_model

In [ ]:
grd = ensemble.GradientBoostingClassifier(n_estimators=10)

In [ ]:
grd_enc = preprocessing.OneHotEncoder()

In [ ]:
grd_lm = linear_model.LogisticRegression()

In [ ]:
grd.fit(x, y)

In [ ]:
grd_enc.fit(gb.apply(x)[:, :, 0])

In [ ]:
grd_lm.fit(grd_enc.transform(gb.apply(x)[:, :, 0]), y)

In [ ]:
y_pred_grd_lm = grd_lm.predict_proba(grd_enc.transform(gb.apply(xt)[:, :, 0]))[:, 1]


In [ ]:
fpr_grd_lm, tpr_grd_lm, _ = roc_curve(y, y_pred_grd_lm)